In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

from pprint import pprint
from warnings import catch_warnings, simplefilter

import random
import numpy as np
import csv
import os
import ast

def read(fname):
    with open(fname, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)
        for entry, in reader:
            yield entry

def read_folder(folder):
    dataset = []
    files = sorted(file for file in os.listdir(folder) if 'task' in file)
    for i, file in enumerate(files):
        for code in list(read(f'{folder}/{file}')):
            dataset.append([code, i])
    return dataset

class Collector(ast.NodeVisitor):
    def __init__(self):
        self.nodes = []
    
    def visit(self, node):
        name = type(node).__name__
        self.nodes.append(name)
        super().visit(node)

def vectorize(codes):
    nodes = set()
    nsets = []
    for code in codes:
        tree = ast.parse(code)
        visitor = Collector()
        visitor.visit(tree)
        nodes.update(visitor.nodes)
        nsets.append(visitor.nodes)
    nodes = list(nodes)
    print(f'AST hist dim: {len(nodes)}')
    vecs = []
    for nset in nsets:
        total = len(nset)
        vec = []
        for node in nodes:
            this = len([i for i in nset if node == i])
            vec.append(this / total)
        vecs.append(vec)
    return vecs

def dataset(folder='data', count=2000):
    random.seed(42)
    programs = random.sample(read_folder(folder), count)
    print(f'Total programs: {len(programs)}')
    vectors = vectorize([code for code, _ in programs])
    x = np.array(list(vectors), dtype=np.float32)
    y = np.array([klass for _, klass in programs], dtype=int).reshape((len(programs), 1))
    return None, x, y

def score(grid_search, scorers):
    required = ['fit_time', 'score_time']
    output = dict()
    for scorer in [f'test_' + s for s in scorers] + required:
        means = grid_search.cv_results_[f'mean_{scorer}']
        stds = grid_search.cv_results_[f'std_{scorer}']
        mean = np.max(means)
        std = np.mean(stds)
        key = scorer if scorer in required else scorer[5:]
        output[key] = (round(mean, 3), round(std, 3))
    return dict(sorted(output.items()))

def find(x, y, estimator, params, verbose=2):
    keys = ['precision_weighted', 'recall_weighted', 'f1_weighted',
            'precision_macro', 'recall_macro', 'f1_macro',
            'accuracy']
    grid_search = GridSearchCV(
        scoring=dict((key, key) for key in keys),
        estimator=estimator(),
        param_grid=params,
        refit='f1_macro',
        verbose=verbose,
        cv=5)
    with catch_warnings():
        simplefilter('ignore')
        grid_search.fit(x, y)
    print()
    pprint(grid_search.best_params_)
    pprint(score(grid_search, keys))
    return grid_search

def statistics(classifier, parameters, *limits):
    keys, scores = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], []
    for count in range(*limits):
        _, X, Y = dataset(count=count)
        model = find(X, Y, classifier, parameters)
        scores.append(score(model, keys))
    pprint(scores, width=400)
    return scores

def describe(dataset):
    H, X, Y = dataset(count=100)
    print(f'Components: {len(X[0])}.')
    print(f'Non-zero components: {len(np.where(X[0] != 0)[0])}.')
    print(f'Zero components: {len(np.where(X[0] == 0)[0])}.')

describe(dataset)

Total programs: 100
AST hist dim: 75
Components: 75.
Non-zero components: 19.
Zero components: 56.


**Plots 100-1000**

In [3]:
stats = statistics(KNeighborsClassifier, {
    'n_neighbors': [4],
    'weights': ('distance',),
    'metric': ('manhattan',)
}, 100, 1000 + 100, 100)

Total programs: 100
AST hist dim: 75
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.6s
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s

{'metric': 'manhattan', 'n_neighbors': 4, 'weights': 'distance'}
{'accuracy': (0.84, 0.066),
 'f1_macro': (0.798, 0.065),
 'f1_weighted': (0.823, 0.066),
 'fit_time': (0.003, 0.005),
 'precision_macro': (0.822, 0.058),
 'precision_weighted': (0.852, 0.054),
 'recall_macro': (0.819, 0.065),
 'recall_weighted': (0.84, 0.066),
 'score_time': (0.157, 0.28)}
Total programs: 200
AST hist dim: 82
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..metric=manhattan, n_neighbors=4, we

In [4]:
stats = statistics(SVC, {
    'C': [30],
    'kernel': ('rbf',),
}, 100, 1000 + 100, 100)

Total programs: 100
AST hist dim: 75
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s

{'C': 30, 'kernel': 'rbf'}
{'accuracy': (0.86, 0.058),
 'f1_macro': (0.814, 0.067),
 'f1_weighted': (0.847, 0.064),
 'fit_time': (0.005, 0.006),
 'precision_macro': (0.831, 0.066),
 'precision_weighted': (0.869, 0.066),
 'recall_macro': (0.835, 0.066),
 'recall_weighted': (0.86, 0.058),
 'score_time': (0.028, 0.028)}
Total programs: 200
AST hist dim: 82
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[C

In [5]:
stats = statistics(RandomForestClassifier, {
    'n_estimators': [300],
    'max_depth': [40],
}, 100, 1000 + 100, 100)

Total programs: 100
AST hist dim: 75
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.4s
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.3s
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.3s
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.3s
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.3s

{'max_depth': 40, 'n_estimators': 300}
{'accuracy': (0.9, 0.045),
 'f1_macro': (0.873, 0.069),
 'f1_weighted': (0.887, 0.052),
 'fit_time': (0.392, 0.035),
 'precision_macro': (0.89, 0.064),
 'precision_weighted': (0.91, 0.047),
 'recall_macro': (0.887, 0.062),
 'recall_weighted': (0.9, 0.045),
 'score_time': (0.034, 0.003)}
Total programs: 200
AST hist dim: 82
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .....................max_depth=40, n_estimators=300; total time=  

In [6]:
stats = statistics(MLPClassifier, {
    'activation': ['relu'],
    'learning_rate_init': [0.001],
}, 100, 1000 + 100, 100)

Total programs: 100
AST hist dim: 75
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.3s
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s

{'activation': 'relu', 'learning_rate_init': 0.001}
{'accuracy': (0.52, 0.108),
 'f1_macro': (0.339, 0.085),
 'f1_weighted': (0.407, 0.115),
 'fit_time': (0.208, 0.08),
 'precision_macro': (0.314, 0.093),
 'precision_weighted': (0.374, 0.12),
 'recall_macro': (0.43, 0.069),
 'recall_weighted': (0.52, 0.108),
 'score_time': (0.004, 0.005)}
Total programs: 200
AST hist dim: 82
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..........activation=relu, learning_rate_init=0.001;